# init web driver

In [2]:
from selenium import webdriver
from pathlib import Path
import os

In [9]:
# root = "/media/ywatcher/ExtDisk1/Files/py/archiv"
driver_root = os.path.join(root, "driver")
drivers = {
    "chrome" : os.path.join(
        driver_root, 
        "chromedriver-linux64",
        # "/home/ywatcher/.cache/selenium/chromedriver/linux64/118.0.5993.70",
        "chromedriver"),
        
    "firefox" : os.path.join(
        driver_root, "geckodriver-v0.33.0-linux64", "geckodriver"),
    "edge": os.path.join(
        driver_root, "edgedriver_linux64", "msedgedriver")
}
# get drivers from https://googlechromelabs.github.io/chrome-for-testing/

In [10]:
t = "chrome"
# t = "firefox"
# t = "edge"
print(drivers[t])
print(os.path.exists(drivers[t]))
print(Path(drivers[t]).is_file())

/media/ywatcher/ExtDisk1/Files/Packages/web_archiv/driver/chromedriver-linux64/chromedriver
True
True


In [11]:
driver = webdriver.Chrome(
        drivers["chrome"]
        #executable_path=drivers["chrome"]
        # service=service,
        # options =chromeOptions
    )

In [12]:
q_id = 625769474

get url


In [205]:
def close_button(driver):
    close_buttons = driver.find_elements_by_xpath(
        #"Button Modal-closeButton Button--plain"
        '//button[@class="Button Modal-closeButton Button--plain"]'
    )
    assert len(close_buttons)==1
    close_buttons[0].click()
    
def expand_question(driver):
    expand_button = None
    all_buttons = driver.find_element_by_xpath(
        '//div[@class="QuestionHeader-content"]'
    ).find_element_by_xpath(
        '//div[@class="QuestionRichText QuestionRichText--expandable QuestionRichText--collapsed"]'
    ).find_elements_by_xpath("//button")
    assert all_buttons[5].text=='显示全部'
    expand_button = all_buttons[5]
    # for b in all_buttons:
    #     b.text
    #     if b.text == '显示全部':
    #         print(b.text)
    #         expand_button = b
    #     break
    expand_button.click()

In [3]:
from util.repo import database_root, root, driver_root
db_test = os.path.join(database_root, "test.1.db")

In [36]:
close_buttons[0].

'button'

In [42]:
source = driver.page_source

In [48]:
from multiprocessing import Process, Queue
import time
import sys

In [171]:
import os.path
from urllib.request import urlopen
from typing import List
import ast
# 导入BeautifulSoup
from bs4 import BeautifulSoup as bf
from bs4.element import Tag
from util.repo import root
import xmljson


def zhihu_question_url(question_id) -> str:
    return "https://www.zhihu.com/question/{}".format(question_id)


def zhihu_answer_url(answer_id) -> str:
    return "https://www.zhihu.com/answer/{}".format(answer_id)

In [300]:
class AnswerParser:
    def __init__(
        self, 
        queue_get_task:Queue, 
        queue_put_result:Queue,
        driver,
        logger=None
    ):
        self.queue_get_task = queue_get_task
        self.queue_put_result = queue_put_result
        self.driver = driver
        self.logger = logger
        
        
    def parse(self, answer_id, question_id=None):
        if self.logger is not None:
            self.logger.log(
                f"parsing answer with id={answer_id}")
        url = zhihu_answer_url(answer_id=answer_id)
        try: 
            self.driver.get(url)
            close_button(self.driver)
        except ElementNotVisibleException :
            print("try again")
            self.driver.get(url)
            close_button(self.driver)
        if question_id is None:
            question_id = "" # FIXME: get question id from web element
        content_blocks = self.driver.find_elements_by_xpath(
            '//div[@class="QuestionAnswer-content"]'
        )
        assert len(content_blocks)==1
        content_block = content_blocks[0].find_element_by_xpath(
            '//div[@class="QuestionAnswer-content"]' # FIXME
        ).get_attribute("outerHTML")
        
        parsed_answer_info = self.parse_content_block(
            html_text=content_block
        )
        self.queue_put_result.put(AnswerBriefInfo(
            question_id=question_id,
            answer_id=answer_id,
            **parsed_answer_info
        ))
        if False: # FIXME
            self.get_comments()
                
        
        
    @staticmethod
    def parse_content_block(html_text:str) -> Dict[str,str]:
        obj = bf(html_text, 'html.parser')
        content = obj.find('div', class_='RichContent-inner').find('span').contents
        author = None  # TODO
        upvoteCount = obj.find('meta', itemprop="upvoteCount").get("content")
        dateCreated=obj.find('meta', itemprop="dateCreated").get("content")
        dateModified = obj.find('meta', itemprop="dateModified").get("content")
        commentCount = obj.find('meta', itemprop='commentCount').get("content")
        return {
            "author":author,
            "content":content, 
            "upvoteCount":upvoteCount, 
            "dateCreated":dateCreated, 
            "dateModified":dateModified, 
            "commentCount":commentCount
        }

    def get_comments(self):
        pass
    
    def start_parsing(self):
        while True:
            task = self.queue_get_task.get(True,2)
            if isinstance(task, TaskStopEvent):
                self.queue_get_task.put(task)
                break
            elif isinstance(task, AnswerParseTask):
                self.parse(
                    answer_id=task.answer_id,
                    question_id=task.question_id
                )
            else:
                raise Exception
        

In [292]:
class QuestionParser:
    def __init__(
        self, 
        queue_put_result: Queue,
        queue_put_task: Queue,
        driver,
        logger=None
    ):
        self.queue_put_result = queue_put_result
        self.queue_put_task = queue_put_task
        self.driver = driver
        self.logger = logger
        
    def parse(self, question_id):
        if self.logger is not None:
            self.logger.log(
                f"parsing question with id={question_id}")
        url = zhihu_question_url(question_id)
        try: 
            self.driver.get(url)
            close_button(self.driver)
        except ElementNotVisibleException :
            print("try again")
            self.driver.get(url)
            close_button(self.driver)
        expand_question(self.driver)
        answerCount = self.driver.find_element_by_xpath(
            '//div[@class="QuestionPage"]/meta[@itemprop="answerCount"]'
        ).get_attribute("content")
        dateCreated = self.driver.find_element_by_xpath(
            '//div[@class="QuestionPage"]/meta[@itemprop="dateCreated"]'
        ).get_attribute("content")
        dateModified = self.driver.find_element_by_xpath(
            '//div[@class="QuestionPage"]/meta[@itemprop="dateModified"]'
        ).get_attribute("content")
        
        content_blocks = self.driver.find_elements_by_xpath(
            '//div[@class="QuestionHeader"]'
        )
        # assert len(content_blocks)==1
        content_block = content_blocks[0].get_attribute("outerHTML")
        # parse content(rich text) and title
        parsed_question_info = self.parse_question_block(content_block)
        self.queue_put_result.put( QuestionBriefInfo(
            question_id=question_id,
            answerCount=answerCount,
            dateCreated=dateCreated,
            dateModified=dateModified,
            **parsed_question_info
        ))
        self.put_answer_tasks_to_queue(question_id)
        
        
    @staticmethod
    def parse_question_block(html_text:str):
        obj = bf(html_text, 'html.parser')
        title = obj.find('h1', class_="QuestionHeader-title").text
        content = obj.find(
            'div', 
            class_="QuestionRichText QuestionRichText--expandable"
        ).find('span').contents
        
        return {
            "title": title,
            "content" : content
        }
    
    def put_answer_tasks_to_queue(self, question_id):
        # FIXME: scroll
        question_answers = self.driver.find_elements_by_xpath(
            '//div[@class="ContentItem AnswerItem"]'
        )
        # TODO: scroll and catch new items loaded
        for q in question_answers:
            self.queue_put_task.put(
                AnswerParseTask(
                    question_id=question_id,
                    answer_id=q.get_attribute("name")
                )
            )
        self.queue_put_task.put(TaskStopEvent())
        
            
            
        
            
        

In [290]:
from dataclasses import dataclass
from typing import Union,Dict
@dataclass
class AnswerBriefInfo:
    question_id: str
    answer_id:str
    author:str
    content: str # html like
    upvoteCount: Union[int,str]
    dateCreated : str
    dateModified:str
    commentCount:str
  
    
@dataclass
class CommentBriefInfo:
    pass


@dataclass
class QuestionBriefInfo:
    question_id:str
    title: str
    content: str
    answerCount: str
    dateCreated : str
    dateModified:str

@dataclass
class AnswerParseTask:
    question_id:Union[int, str]
    answer_id: Union[int, str]
    
class TaskStopEvent:
    pass

In [231]:
new_driver.find_elements_by_xpath(
            '//div[@class="QuestionHeader"]'
        )[0].get_attribute("outerHTML")

'<div class="QuestionHeader"><div class="QuestionHeader-content"><div class="QuestionHeader-main"><div class="QuestionHeader-tags"><div class="QuestionHeader-topics"><div class="Tag QuestionTopic css-1s3a4zw" data-za-detail-view-path-module="TopicItem" data-za-extra-module="{&quot;card&quot;:{&quot;content&quot;:{&quot;type&quot;:&quot;Topic&quot;,&quot;token&quot;:&quot;19550706&quot;}}}"><span class="Tag-content"><a class="TopicLink" href="//www.zhihu.com/topic/19550706" target="_blank"><div class="css-1gomreu">纪录片</div></a></span></div><div class="Tag QuestionTopic css-1s3a4zw" data-za-detail-view-path-module="TopicItem" data-za-extra-module="{&quot;card&quot;:{&quot;content&quot;:{&quot;type&quot;:&quot;Topic&quot;,&quot;token&quot;:&quot;20142632&quot;}}}"><span class="Tag-content"><a class="TopicLink" href="//www.zhihu.com/topic/20142632" target="_blank"><div class="css-1gomreu">文化自信</div></a></span></div><div class="Tag QuestionTopic css-1s3a4zw" data-za-detail-view-path-module=

In [40]:



class Author:
    pass


class Question:
    def __init__(
            self,
            question_id: int = None,
            title: str = None,
            meta: List[Tag] = None,
            answers=None
    ):
        self.title = title
        self.question_id = question_id
        self.answers = answers
        self.meta = meta

    def parse(self, recurse: bool, text=None):
        print("parsing question")
        if text is None:
            url = zhihu_question_url(self.question_id)
            html = urlopen(url)
            # 获取的html内容是字节，将其转化为字符串
            html_text = bytes.decode(html.read())
        else:
            html_text = text
        # 用BeautifulSoup解析html
        obj = bf(html_text, 'html.parser')
        if self.title is None:
            self.title = obj.head.title.text
        question_info = obj.find('div', class_='QuestionPage')
        if self.question_id is None:
            # for example data-za-extra-module='{"card":{"content":{"type":"Question","token":"532925796"}}}'
            data_za_extra_module_dict = ast.literal_eval(question_info.get('data-za-extra-module'))
            self.question_id = int(data_za_extra_module_dict["card"]["content"]["token"])
        print("\twith question id={}".format(self.question_id))
        print("\ttitle:{}".format(self.title))
        if self.meta is None:
            self.meta = question_info.find_all("meta")
        if self.answers is None:
            # self.list = obj.find_all('div', class_="List-item")
            answer_blocks = obj.find_all('div', class_="ContentItem AnswerItem")
            self.answers = [
                parse_answer_block_from_question(answer, recurse=recurse)
                for answer in answer_blocks
            ]


class Answer:
    def __init__(
            self,
            data_za_index: int,
            author_name: str,
            answer_id: int,
            itemprop: str,
            meta: List[Tag],
            content=None,
            comments=None
    ):
        self.data_za_index = data_za_index
        self.author_name = author_name
        self.answer_id = answer_id
        self.itemprop = itemprop
        self.content = content
        self.comments = comments
        self.meta = meta

    def get_content_and_comment(self):
        print("\t\tparsing answer with id={}".format(self.answer_id))
        # 发出请求，获取html
        url = zhihu_answer_url(self.answer_id)
        html = urlopen(url)
        # 获取的html内容是字节，将其转化为字符串
        html_text = bytes.decode(html.read())
        # 用BeautifulSoup解析html
        obj = bf(html_text, 'html.parser')
        # self.p = obj.find('p')
        # self.content: a list of paragraphs and figures
        self.content = obj.find('div', class_='RichContent-inner').find('span').contents

    def __repr__(self):
        return "Answer(id={})".format(self.answer_id)


class Paragraph:
    def __init__(self, text):
        self.text = text


def parse_answer_block_from_question(answer_block: Tag, recurse: bool):
    index = int(answer_block.get("data-za-index"))
    data_zop_dict = ast.literal_eval(answer_block.get("data-zop"))
    author_name = data_zop_dict["authorName"]
    answer_id = data_zop_dict["itemId"]
    itemprop = answer_block.get("itemprop")
    meta = answer_block.find_all("meta")
    answer = Answer(
        data_za_index=index,
        author_name=author_name,
        answer_id=answer_id,
        itemprop=itemprop,
        meta=meta,
    )
    if recurse:
        answer.get_content_and_comment()
    return answer



In [45]:
question = Question()
question.parse(recurse=True, text=source)

parsing question
	with question id=625769474
	title:如何评价纪录片《当马克思遇见孔夫子》? - 知乎
		parsing answer with id=3252328928
		parsing answer with id=3249748730
		parsing answer with id=3252760617
		parsing answer with id=3250133829
		parsing answer with id=3249016418
		parsing answer with id=3257691327
		parsing answer with id=3250103826
		parsing answer with id=3256584908
		parsing answer with id=3261422855
		parsing answer with id=3261230756
		parsing answer with id=3250339752
		parsing answer with id=3256551441
		parsing answer with id=3256494643
		parsing answer with id=3254833311
		parsing answer with id=3250371750
		parsing answer with id=3255212000
		parsing answer with id=3263193884
		parsing answer with id=3259707086
		parsing answer with id=3255824921
		parsing answer with id=3265831161
		parsing answer with id=3263632092
		parsing answer with id=3262183598
		parsing answer with id=3260397226
		parsing answer with id=3266453379
		parsing answer with id=3266248640
		parsing answer with i

In [47]:
print(question.answers[0].content)

[<p data-first-child="" data-pid="kkerF_7K">这部片子成功达到了和春晚一样的高度</p>, <p data-pid="7ccIYuvm">不管是贴吧，b乎，抖音，微博，乃至小红书</p>, <p data-pid="bVub0cca">不管是左，中还是右</p>, <p data-pid="n_aDbji-">不管是神神还是兔兔</p>, <p data-pid="zuE2ZdNq">不管ip在哪</p>, <p data-pid="iN6Dbmjx">甚至性别都与平时不同</p>, <p data-pid="Kadlnu1W">十分罕见的一致认为这玩意是个垃圾中的垃圾</p>]


In [173]:
url = zhihu_question_url(
    q_id
)
html = urlopen(url)
html_text = bytes.decode(html.read())
with open("save.txt","w") as f:
    f.write(html_text)

In [153]:
class Q:
    def __init__(self):
        self.l = []
        
    def put(self,o):
        self.l.append(o)

task = Q()
result = Q()

In [159]:
# for answer in answer page:
new_driver =  webdriver.Chrome(
        drivers["chrome"]
        #executable_path=drivers["chrome"]
        # service=service,
        # options =chromeOptions
    )

In [52]:
new_driver.get(zhihu_answer_url(3259178660))

In [301]:
result = Queue()
task = Queue()
question_parser = QuestionParser(
    queue_put_result=result,
    queue_put_task=task,
    driver=new_driver, logger=None)
answer_parser = AnswerParser(
    queue_get_task=task,
    queue_put_result=result,
    driver=new_driver, logger=None)

In [234]:
b = answer_parser.parse(
    # 3259178660
    # 3259133218
    3250133829
)

In [302]:
b = question_parser.parse(
    q_id
)

In [303]:
answer_parser.start_parsing()

In [285]:
for _task in task.l:
    answer_parser.parse(
        answer_id=_task.answer_id,
        question_id=_task.question_id
    )

In [253]:
b = new_driver.find_element_by_xpath(
            '//div[@class="QuestionPage"]/meta[@itemprop="answerCount"]'
        )

In [287]:
result.l

[QuestionBriefInfo(question_id=625769474, title='如何评价纪录片《当马克思遇见孔夫子》?', content=[<p data-first-child=""></p>, <figure data-size="normal"><div><img class="origin_image zh-lightbox-thumb" data-original="https://pic3.zhimg.com/v2-657d0837f091bb051f08d45ddbe07422_r.jpg" data-rawheight="1080" data-rawwidth="1917" data-size="normal" src="https://pic3.zhimg.com/v2-657d0837f091bb051f08d45ddbe07422_b.png" width="1917"/></div></figure>, <p>图源b站</p>], answerCount='68', dateCreated='2023-10-12T04:50:29.000Z', dateModified='2023-10-12T04:50:29.000Z'),
 AnswerBriefInfo(question_id=625769474, answer_id='3252328928', author=None, content=[<p data-first-child="" data-pid="kkerF_7K">这部片子成功达到了和春晚一样的高度</p>, <p data-pid="7ccIYuvm">不管是贴吧，b乎，抖音，微博，乃至小红书</p>, <p data-pid="bVub0cca">不管是左，中还是右</p>, <p data-pid="n_aDbji-">不管是神神还是兔兔</p>, <p data-pid="zuE2ZdNq">不管ip在哪</p>, <p data-pid="iN6Dbmjx">甚至性别都与平时不同</p>, <p data-pid="Kadlnu1W">十分罕见的一致认为这玩意是个垃圾中的垃圾</p>], upvoteCount='50', dateCreated='2023-10-16T11:44:25.000Z

In [251]:
b = new_driver.find_element_by_xpath(
            '//div[@class="QuestionPage"]')



In [252]:
b.get_attribute("")

In [70]:
content[0].find_element_by_xpath(
    [

[<selenium.webdriver.remote.webelement.WebElement (session="6b3166495063ab3d5fe5993f7e34faa6", element="406BF7D1B846F45993A8049B861C71F2_element_252")>]

In [58]:
len(content)

1

In [59]:
new_driver.close()


In [116]:
answer_item = b

In [122]:
b.get_attribute("data-zop")

In [95]:
answer_item = b[0].find_element_by_xpath(
    '//div[@class="QuestionAnswer-content"]'
)

In [97]:
answer_item.get_attribute("data-zop")

In [117]:
d = answer_item.find_element_by_xpath(
    '//meta[@itemprop="dateCreated"]'
)

In [120]:
h = d.get_attribute("outerHTML")

In [121]:
h

'<meta itemprop="dateCreated" content="2023-10-12T04:50:29.000Z">'

In [123]:
type(b)

selenium.webdriver.remote.webelement.WebElement

In [164]:
t = new_driver.find_element_by_xpath(
    '//h1[@class="QuestionHeader-title"]'
)

In [166]:
t.get_attribute("outerHTML")

'<h1 class="QuestionHeader-title">如何评价纪录片《当马克思遇见孔夫子》?</h1>'

<selenium.webdriver.remote.webelement.WebElement (session="26433fb6b10e5939807f4856a8755b89", element="4BA0D500254B4C3BEF417E604CDAF8D3_element_193")>